In [62]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [63]:
#initialize a dictionary of boxscores which will be saved as pandas df's'
dict_boxscores = {}
year = '2017'
month = '4'
day = '15'
print 'http://www.basketball-reference.com/boxscores/?month={0}&day={1}&year={2}'.format(month,day,year)

http://www.basketball-reference.com/boxscores/?month=4&day=15&year=2017


In [124]:
def box_2017():
    year = '2017'
    months = ['1''2','3','4']
    days = [str(i) for i in range(32)]
    for month in months:
        for day in days:  
            response = requests.get('http://www.basketball-reference.com/boxscores/?month={0}&day={1}&year={2}'.format(month,day,year))
            soup = BeautifulSoup(response.content, 'lxml')
            tables = soup.find_all('div',{'class':'game_summary expanded nohover'})
            for table in tables:
                box = []
                teams = {}
                teams['away'] = table.find_all('table')[1].find('tbody').find_all('tr')[0].find('td').text
                teams['away_score'] = table.find('table').find_all('tr')[0].find_all('td')[1].text
                teams['home'] = table.find_all('table')[1].find('tbody').find_all('tr')[1].find('td').text
                teams['home_score'] = table.find('table').find_all('tr')[1].find_all('td')[1].text
                teams['away_r'] = table.find('table').find('tr')['class'][0]
                teams['home_r'] = table.find('table').find_all('tr')[1]['class'][0]
                response = requests.get('http://www.basketball-reference.com{0}'.format(table.find('p').find('a')['href']))
                soup = BeautifulSoup(response.content, 'lxml')
                for athlete in soup.find_all('table')[0].find('tbody').find_all('tr'):
                    if athlete.find('th').text.strip() not in ["Reserves","Team Totals"]:
                        player = {}
                        player['name'] = athlete.find('th').text
                        cells = athlete.find_all('td')

                        player['minutes'] = 'DNP'
                        player['team_score']=teams['away_score']
                        player['opp_score']=teams['home_score']
                        player['Team'] = teams['away']
                        player['Date'] = month + '/'+day+'/'+year
                        player['Opponent'] = teams['home']
                        player['result'] = teams['away_r']
                        if len(cells)>1:
                            player['minutes'] = cells[0].text
                            player['FG'] = cells[1].text
                            player['FGA'] = cells[2].text
                            player['3P'] = cells[4].text
                            player['3PA'] = cells[5].text
                            player['FT'] = cells[7].text
                            player['FTA'] = cells[8].text
                            player['ORB'] = cells[10].text
                            player['TRB'] = cells[12].text
                            player['AST'] = cells[13].text
                            player['STL'] = cells[14].text
                            player['BLK'] = cells[15].text
                            player['TOV'] = cells[16].text
                            player['PF'] = cells[17].text
                            player['PTS'] = cells[18].text
                            player['+/1'] = cells[19].text
                        box.append(player)
                for athlete in soup.find_all('table')[2].find('tbody').find_all('tr'):
                    if athlete.find('th').text.strip() not in ["Reserves","Team Totals"]:
                        player = {}
                        player['name'] = athlete.find('th').text
                        cells = athlete.find_all('td')
                        player['team_score']=teams['home_score']
                        player['opp_score']=teams['away_score']
                        player['minutes'] = 'DNP'
                        player['Team'] = teams['home']
                        player['Date'] = month + '/'+day+'/'+year
                        player['Opponent'] = teams['away']
                        player['result'] = teams['home_r']
                        if len(cells)>1:
                            player['minutes'] = cells[0].text
                            player['FG'] = cells[1].text
                            player['FGA'] = cells[2].text
                            player['3P'] = cells[4].text
                            player['3PA'] = cells[5].text
                            player['FT'] = cells[7].text
                            player['FTA'] = cells[8].text
                            player['ORB'] = cells[10].text
                            player['TRB'] = cells[12].text
                            player['AST'] = cells[13].text
                            player['STL'] = cells[14].text
                            player['BLK'] = cells[15].text
                            player['TOV'] = cells[16].text
                            player['PF'] = cells[17].text
                            player['PTS'] = cells[18].text
                            player['+/1'] = cells[19].text
                        box.append(player)
                game_id = player['Date'] + '_'+player['Opponent']+'_'+player['Team']
                dict_boxscores[game_id] = pd.DataFrame(box)
    return dict_boxscores
boxes_2017 = box_2017() 


In [126]:
boxes_2017['3/15/2017_Memphis_Chicago']


,+/1,3P,3PA,AST,BLK,Date,FG,FGA,FT,FTA,...,PTS,STL,TOV,TRB,Team,minutes,name,opp_score,result,team_score
0,+8,3,6,7,1,3/15/2017,10,21,4,5,...,27,0,0,6,Memphis,37:20,Marc Gasol,91,winner,98
1,+19,5,8,7,0,3/15/2017,8,12,6,6,...,27,1,4,9,Memphis,36:09,Mike Conley,91,winner,98
2,+11,1,4,2,1,3/15/2017,3,7,0,0,...,7,0,1,3,Memphis,29:11,Vince Carter,91,winner,98
3,+15,0,1,3,0,3/15/2017,4,10,2,2,...,10,0,0,6,Memphis,23:52,Tony Allen,91,winner,98
4,+12,0,2,1,1,3/15/2017,2,5,0,0,...,4,1,1,7,Memphis,23:06,JaMychal Green,91,winner,98
5,-5,0,0,3,0,3/15/2017,4,14,0,0,...,8,0,0,9,Memphis,24:54,Zach Randolph,91,winner,98
6,-12,0,2,1,0,3/15/2017,1,3,3,4,...,5,0,1,4,Memphis,24:24,James Ennis,91,winner,98
7,-2,2,7,0,0,3/15/2017,2,8,0,0,...,6,0,0,3,Memphis,19:17,Troy Daniels,91,winner,98
8,-10,0,1,2,0,3/15/2017,0,2,0,0,...,0,0,3,1,Memphis,11:07,Andrew Harrison,91,winner,98
9,-1,0,0,0,0,3/15/2017,2,3,0,1,...,4,0,1,4,Memphis,10:40,Brandan Wright,91,winner,98


In [66]:
x = []
for h in x:
    print h

In [75]:
df.shape

(26, 19)

In [79]:
df['Team'].nunique()

8